In [ ]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

f_data_loading <- "./data_loading.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_data_loading, '--to=script', '--stdout'), stdout = TRUE)))

In [ ]:
SPL_STATS_DIR = "../../output/cfspi/samples/results/stats/"
sample_nm <- c()
step_nm <- c()
for (file in list.files(SPL_STATS_DIR, pattern = "R1")){
    sample_nm <- c(sample_nm, unlist(strsplit(file, '_'))[1])
    step_nm <- c(step_nm, paste0(unlist(strsplit(file, '_'))[3], unlist(strsplit(file, '_'))[4]))
}
mt <- matrix(ncol = length(unique(sample_nm)), nrow = length(unique(step_nm)))
colnames(mt) <- unique(sample_nm)
rownames(mt) <- unique(step_nm)
for (file in list.files(SPL_STATS_DIR)){
    sample <- unlist(strsplit(file, '_'))[1]
    step <- paste0(unlist(strsplit(file, '_'))[3], unlist(strsplit(file, '_'))[4])
    line_count <- read.csv(file = paste0(SPL_STATS_DIR,file), header=FALSE, sep = "\t", stringsAsFactors=FALSE)
    mt[step, sample] <- line_count[1,1]
}

In [ ]:
percentage_PCRdupl = data.frame(sample_id = colnames(mt), uniq_percentage = (mt[1,]-mt[2,])/mt[1,]*100)
percentage_PCRdupl_meta <- merge(x = samples_metadata, y = percentage_PCRdupl, by.x = "sample_id", by.y = "sample_id") 

In [ ]:
percentage_PCRdupl_meta %>% 
    filter(sample_type != "ds-wcDNA BAL") %>%  
    filter(type == "ssLP") %>% 
    filter(origin != "controls") %>% 
    #group_by(sample_type) %>% 
    summarise(Q1 = quantile(uniq_percentage, probs = 0.25),
              Q2 = quantile(uniq_percentage, probs = 0.50),
              Q3 = quantile(uniq_percentage, probs = 0.75))